## Tweet Generation

In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn import metrics
from collections import defaultdict
import matplotlib.pyplot as plt
import glob


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics.cluster import contingency_matrix

## Import libraries
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation
import matplotlib.pyplot as plt

In [2]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import random
import pandas as pd
import numpy as np
import re, json, datetime, pytz
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from collections import Counter
import nltk

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import pytz
import datetime

pst_tz = pytz.timezone('America/Los_Angeles')

In [4]:
from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification(model_type="DISTILBERT", 
                                       model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

03/14/2022 00:14:02 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [5]:
from transformers import pipeline
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

In [6]:
def transformer_sentiment(tweet):
    
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score
    

In [7]:
def translate_and_transformer_sentiment(tweet):
    
    tweet = pipe(tweet, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))
    tweet = tweet[0]['generated_text']
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score

In [106]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
#     return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", tweet).split())
    return ' '.join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", tweet).split())

In [107]:
from_time = datetime.datetime.strptime('2015-02-01 15:00:00', '%Y-%m-%d %H:%M:%S')
to_time =  datetime.datetime.strptime('2015-02-02 10:00:00', '%Y-%m-%d %H:%M:%S')


def get_data(text, min_retweet=10):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            time_now = datetime.datetime.fromtimestamp(tweet['citation_date'])
            retweet_now = tweet['metrics']['citations']['total']
            if time_now > from_time and time_now < to_time and retweet_now >= min_retweet and tweet['tweet']['lang'] == 'en': 
                retweet_count.append(tweet['metrics']['citations']['total']) 
                fav_count.append(tweet['tweet']['favorite_count'])
                followers.append(tweet['author']['followers'])
                time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
                place.append(tweet['tweet']['user']['location'])
                cleaned_tweet = clean_tweet(tweet['title'])
                senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                sentiment_polarity.append(senti_polarity)
                sentiment_score.append(senti_score)
                title.append(cleaned_tweet)
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   

In [108]:
def get_player(text, player=['tom', 'brady']):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            if tweet['metrics']['citations']['total'] >= 15:
                tweet_text = tweet['title']
                tweet_text = tweet_text.lower().split()
                for word in player: 
                    if word in tweet_text:
                        title.append(tweet['title'])
                        retweet_count.append(tweet['metrics']['citations']['total']) 
                        fav_count.append(tweet['tweet']['favorite_count'])
                        followers.append(tweet['author']['followers'])
                        time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
                        place.append(tweet['tweet']['user']['location'])
                        cleaned_tweet = clean_tweet(tweet['title'])
                        if not tweet['tweet']['lang'] == 'en':
                            senti_polarity, senti_score = translate_and_transformer_sentiment(cleaned_tweet)
                            sentiment_polarity.append(senti_polarity)
                            sentiment_score.append(senti_score)
                        else:
                            senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                            sentiment_polarity.append(senti_polarity)
                            sentiment_score.append(senti_score)
                        break
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   

In [116]:
file1 = open('tweets_#nfl.txt', 'r')
lines = file1.readlines()

In [118]:
count = 0
for line in lines:
    json_object = json.loads(line)
    if json_object['metrics']['citations']['total'] >= 30 and json_object['tweet']['lang'] == 'en':
        count = count + 1
print(count)

453


In [120]:
df = get_data('tweets_#nfl', 30)

/home/shivam/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [121]:
len(df)

108

In [122]:
df

,title,retweet_count,favourite_count,followers,time,place,sentiment_polarity,sentiment_score
0,Repping the Patriots NFLSnow nfl rcolvin3,79,0,285.0,2015-02-01 22:32:44-08:00,,-1,-0.994350
1,RT FOR PATRIOTS FAV FOR SEAHAWKS SB 49 nfl SuperBowl SuperBowlXLIX,172,0,68.0,2015-02-01 16:01:46-08:00,SoFlo,-1,-0.742071
2,En route Phoenix Bring on the SuperBowlIXLIX BellaBowl Patriots NFL TomBrady FearlessNikki,115,468,12.0,2015-02-02 04:58:55-08:00,Total Divas on E! & RAW on USA,1,0.995345
3,Happy SuperBowl friskyfriday NFL Pats Seahawks playmate Playboy PlayboyMagSA Playboy NL PlayboyPlus,30,0,814.0,2015-02-02 06:44:39-08:00,Arizona,1,0.996990
4,Just saw a GREAT interview with the Deflated NFL Football on NBC,172,0,2642.0,2015-02-01 17:35:25-08:00,Disney XD,1,0.996231
...,...,...,...,...,...,...,...,...
103,VIDEO watch Malcolm Butler s amazing goal line pick preserving N E s SB49 win NFL,60,0,1379852.0,2015-02-01 20:37:35-08:00,Sunnyvale CA,1,0.999197
104,You want to talk about Montana We can do that But no one should ever again mention Manning in the same breath as Brady Patriots NFL,32,0,1260.0,2015-02-01 20:37:58-08:00,"Florence, Massachusetts",-1,-0.707377
105,Thanks you nfl for a perfect evening Time for sleep Congrats Patriots Thanks for letting me,126,0,110349.0,2015-02-01 23:34:05-08:00,beachersmadhouse.com LA,1,0.999422
106,Simply the Best Tom Brady now stands alone as greatest QB of all time WEEI NFL Patriots,37,0,20233.0,2015-02-02 02:14:45-08:00,Massachusetts,1,0.999777


In [123]:
times = pd.to_datetime(df.time)
grouped = df.groupby([times.dt.date, times.dt.hour]).title.transform(lambda x: '. '.join(x))
grouped = grouped.drop_duplicates()

In [124]:
pd.options.display.max_colwidth = 1000
print(grouped.head(10))

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        Repping the Patriots NFLSnow nfl rcolvin3. Go seahawks Foam Padded tabletcase for ipad kindle etsymntt nfl superbowl integritytt. We re too nice So you can have until Midday to enter our RT comp SuperBowl SuperBowlSunday NFL BeastMode Seahawks AmericanFoo

In [125]:
len(grouped)

14

In [73]:
import transformers

In [129]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [126]:
hour_count = 0
for index, value in grouped.items():
    to_tokenize = value.split(',,')
    summarized = summarizer(to_tokenize , min_length=2, max_length=20.  do_sample=False)
    summ=' '.join([str(i) for i in summarized])
    summ=summ.replace("{","")
    summ=summ.replace("}","")
    summ=summ.replace("'summary_text':","")
    summ=summ.replace("'","")
    summ=summ.replace("''","")
    summ=summ.replace("\x92","")
    hour_count = hour_count+1
    print('Tweet Hour' + str(hour_count) +': ' + summ)
    

Tweet Hour1:   Repping the Patriots NFLSnow nfl rcolvin3. Go seahawks
Tweet Hour2:  " NFL.24x7 NFL SuperBowl XLIX. Cant wait for Super"
Tweet Hour3:   NFL TomBrady FearlessNikki wins Super BowlIXLIX BellaBowl
Tweet Hour4:   NFL Pats Seahawks playmate Playboy PlayboyMagSA Playboy NL PlayboyPlus PlayboyPlus . NFL
Tweet Hour5:   Retweet for a chance to win 100 in SuperBowl swag from jetrules
Tweet Hour6:   The official Selfie of Superbowl 2015 CjMiles SuperBowl 2015 C
Tweet Hour7:   Check out photos from the Patriots SB49 team photo shoot .
Tweet Hour8:   FaceOffTime NFL fans reveal which QB leads their team to SuperBowl win .
Tweet Hour9:   Malcolm Butler s INT to clinch a SBXLIX win for the Patriots NFL. Malcolm
Tweet Hour10:   If you could create your own NFL franchise what would it be called The Portland Perine
Tweet Hour11:   Tom Brady now holds the NFL record for most SuperBowl passing TDs 12 . Beyon
Tweet Hour12:   NewEnglandPatriots beat defending champion SeattleSeahawks 28 24 to

In [130]:
hour_count = 0
for index, value in grouped.items():
    to_tokenize = value.split(',,')
    summarized = summarizer(to_tokenize , min_length=2, max_length=50,  do_sample=False)
    summ=' '.join([str(i) for i in summarized])
    summ=summ.replace("{","")
    summ=summ.replace("}","")
    summ=summ.replace("'summary_text':","")
    summ=summ.replace("'","")
    summ=summ.replace("''","")
    summ=summ.replace("\x92","")
    hour_count = hour_count+1
    print('Tweet Hour' + str(hour_count) +': ' + summ)
    

Tweet Hour1:  RT comp SuperBowl SuperB BowlSunday NFL Beast Mode Seahawks AmericanFootball. Repping the Patriots NFLSnow nfl rcolvin3. Go seahawks Foam Padded tabletcase for ipad kindle etsym


Your max_length is set to 50, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


Tweet Hour2:  SuperBowl XLIX begins tonight at 8 p.m. ET. Seattle Seahawks take on the New England Patriots in SuperBowl 49.


Your max_length is set to 50, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


Tweet Hour3:  BowlIXLIX BellaBowl Patriots NFL TomBrady FearlessNikki. En route Phoenix Bring on the SuperBowl.
Tweet Hour4:  NFL Pats Seahawks playmate Playboy PlayboyMagSA Playboy NL PlayboyPlus. Happy SuperBowl friskyfriday.


...

Tweet Hour7:  Check out photos from the Patriots SB49 team photo shoot. Lets do this Hut 1 hut 2 Superbowl.
Tweet Hour8:   FaceOffTime NFL fans Which QB leads their team to SuperBowl win RT Brady amp Patriots FAV Wilson amp Seahawks.
Tweet Hour9:  Patriots become the sixth franchise in NFL history to win at least four Super Bowls. Malcolm Butler s INT to clinch a SBXLIX win for the Patriots NFL. Jermaine Kearse s INSANE Catch that means absolutely nothing
Tweet Hour10:  Predict the score at the end of the first quarter of SB49 for a prize. Shazam the BigGame to see which idinamenzel songs were Shazamed the most during NFL season.
Tweet Hour11:  Tom Brady now holds the NFL record for most SuperBowl passing TDs 12. Beyonc was still my favorite halftime Sorry nflsuperbowl halftime.


Your max_length is set to 50, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


Tweet Hour12:  NewEnglandPatriots beat defending champion SeattleSeahawks 28 24 to win SuperBowl nfl. Deion is not on the fence on this one nfl SB49.


Your max_length is set to 50, but you input_length is only 31. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


Tweet Hour13:  Tom Brady now stands alone as greatest QB of all time, according to WEEI NFL Patriots.
Tweet Hour14:  BizBash producers review of superbowlXLIX pepsi HypedForHalftime show.


## Commentary Generation

In [132]:
file1 = open('tweets_#superbowl.txt', 'r')
lines = file1.readlines()

In [136]:
from_time = datetime.datetime.strptime('2015-02-01 15:00:00', '%Y-%m-%d %H:%M:%S')
to_time =  datetime.datetime.strptime('2015-02-02 10:00:00', '%Y-%m-%d %H:%M:%S')

count = 0
for line in lines:
    json_object = json.loads(line)
    time_now = datetime.datetime.fromtimestamp(json_object['citation_date'])
    if json_object['metrics']['citations']['total'] >= 150 and json_object['tweet']['lang'] == 'en' and time_now > from_time and time_now < to_time:
        count = count + 1
print(count)

1055


In [137]:
df_superbowl = get_data('tweets_#superbowl', 150)

/home/shivam/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [138]:
len(df_superbowl)

1055

In [139]:
df_superbowl

,title,retweet_count,favourite_count,followers,time,place,sentiment_polarity,sentiment_score
0,SuperBowl snickers stuffed strawberries recipe AND a chocolatesupply basket giveaway ENTER TO win Sweepstakes,200,0,5.0,2015-02-01 17:25:26-08:00,"Austin, TX",1,0.998629
1,A 30 second ad in this year s SuperBowl will set a brand back about 4 5 million,152,0,586.0,2015-02-01 17:26:28-08:00,"New York, NY",-1,-0.990321
2,BellaBowl has begun on WWE com espn nfl SportsCenter Patriots Seahawks SuperBowlXLIX,506,0,15.0,2015-02-01 16:04:18-08:00,Total Divas on E! & RAW on USA,-1,-0.892226
3,Aaron Hernandez murder trial is doubtful for Sunday s SuperBowl,2463,0,212.0,2015-02-01 17:09:20-08:00,"New York, NY",-1,-0.993053
4,Retweet for Patriots Fav for Seahawks SuperBowlXLIX,3102,0,43.0,2015-02-01 16:52:10-08:00,,-1,-0.985823
...,...,...,...,...,...,...,...,...
1050,FAV IF YOU PICKED THE Patriots To Win SuperBowlXLIX Use Code Co For 5 Off GET IT,347,0,179.0,2015-02-02 05:46:35-08:00,LAX-HTX,-1,-0.982248
1051,psyched 2 contribute 2 BizBash producers review of superbowlXLIX pepsi HypedForHalftime show PepsiCo NFL NFL,370,0,397261.0,2015-02-02 07:31:52-08:00,"New York, NY",1,0.678357
1052,A lot going on today Stay tuned for Super Bowl Coin Up TOTW MVP Updates later today UltimateSB SuperBowlXLIX,223,0,128491.0,2015-02-02 08:35:16-08:00,"Orlando, Florida",1,0.994019
1053,All katyperry was missing last night was a sheet of ice SB49 SuperBowl,158,0,61570.0,2015-02-02 08:50:54-08:00,"Colorado Springs, Colo.",-1,-0.998414


In [199]:
times = pd.to_datetime(df_superbowl.time)
grouped = df_superbowl.groupby(pd.Grouper(key='time', axis=0, freq='5min')).title.transform(lambda x: '. '.join(x))
grouped = grouped.drop_duplicates()

In [200]:
len(grouped)

95

In [201]:
pd.options.display.max_colwidth = 10000
print(grouped.head(10))

150                                                                                                                                                                                                                                                                                     Still time to Win a pair of Monster24k headphones RT to enter SuperBowl BeastMode SuperBowlXLIX MoneyLynch. PetrCech and Ivanovic playing American Football at the Seattle Seahawks training facility SB49 SuperBowlSunday. RT if you re watching the Super Bowl Patriots vs Seahawks SB49 SuperBowlSunday. It s our 2nd annual SupernovaSunday During SuperBowl we re posting supernova info sb49. How about our girl carrieunderwood kicking off this Super Bowl Fight SuperBowlXLIX. ISS Research Trivia for your SuperBowlXLIX Patriots Seahawks party Q How big is Space Station A
159                                                                                                                                                     

In [203]:
hour_count = 0
for index, value in grouped.items():
    to_tokenize = value[:3500].split(',,,')
    summarized = summarizer(to_tokenize, min_length=10, max_length=50, do_sample=False)
    summ=' '.join([str(i) for i in summarized])
    summ=summ.replace("{","")
    summ=summ.replace("}","")
    summ=summ.replace("'summary_text':","")
    summ=summ.replace("'","")
    summ=summ.replace("''","")
    summ=summ.replace("\x92","")
    hour_count = hour_count+1
    print('Commentary' + str(hour_count) +': ' + summ)
    

Commentary1:  SuperBowl XLIX takes place on Sunday, February 4th. PetrCech and Ivanovic play American Football at the Seattle Seahawks training facility.
Commentary2:  Million Coin MVP Item Winner announced at Halftime UltimateSB SuperBowlXLIX. SB49 SuperB Bowl2015 SundayFunday.
Commentary3:  Seahawks vs New England Patriots in SuperBowl XLIX LIVE UPDATES SB49. This is gonna be a gas TheBlacklist returns tonight after SuperB BowlXLIX.
Commentary4:  The SuperBowl ads of 2015 Watch them here SB49. EW. Everytime ur girlfriend says I just watch SuperB Bowl for the commercials the inventor of TiVo cries a little.
Commentary5:  The start of the SuperBowl always gives me the chills but this time I think it s wings at this red roof inn bar. Rihanna s chillin SuperB BowlXLIX.
Commentary6:  The footballs have been weighed and measured. So proud of our Airmen representing the AirForce at SuperBowlXLIX. Did you see the AFThunderbirds flyover?
Commentary7:  SuperBowl XLIX begins at 8 p.m. ET. The N

Your max_length is set to 50, but you input_length is only 47. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Commentary8:  SuperBowlXLIX. Will this be the SuperBowl that makes the world finally stop calling soccer football.
Commentary9:  SuperBowl commercial features the National Anthem and Flyby. The commercial also features the GameofWar SuperBowl ad.
Commentary10:  If Seahawks RT this nebula with their team colors SuperBowl SB49. Who are you rooting for this SupernovaSunday? RT for Seahawks Fav for Patriots.
Commentary11:  "TeamGleason is looking to get 100,000 followers on SuperBowl Sunday. Were only 5,000 away."
Commentary12:  The top 10 hottest cheerleaders in SuperBowlXLIX RT Patriots FAV Seahawks. Katy Perry and Lenny Kravitz on stage together. What if the Dodge Neon was a band?
Commentary13:  Good luck to Tom Brady and the New England Patriots in SuperBowlXLIX. Adele Dazeem sings the National Anthem tonight. Kim Kardashian just wants you to see all of her selfies.
Commentary14:  SuperBowl Sunday superbowlcommercials and SuperSoulSunday we are so ready via. katyperry. Retweet for Patr

Your max_length is set to 50, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


Commentary58:   Billboard cover star KatyPerry shined during her SuperBowl XLIX halftime show. SuperB BowlXLIX had to have been booked by HeymanHustle. 19 times katyperry s SuperBOW halftime


...

Commentary92:  Stay tuned for Super Bowl Coin Up TOTW MVP Updates later today UltimateSB SuperBowlXLIX.


Your max_length is set to 50, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


Commentary93:  All katyperry was missing last night was a sheet of ice SB49 SuperBowl.


Your max_length is set to 50, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


Commentary94:  In honor of the Patriots win we will GIVEAWAY these cleats RT and FOLLOW to enter SuperBowl.
Commentary95:  Seahawks defense one of the great defenses of all time stakes its claim with another dominating performance SuperBowl KOD.
